In [38]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import cv2
import scipy

In [2]:
from IPython.display import Audio, display

url_exception = "http://www.wav-sounds.com/movie/austinpowers.wav"
url_exception = "https://www.myinstants.com/media/sounds/roblox-death-sound_1.mp3"


def play_sound(self, etype, value, tb, tb_offset=None):
    self.showtraceback((etype, value, tb), tb_offset=tb_offset)
    display(Audio(url=url_exception, autoplay=True))


get_ipython().set_custom_exc((Exception,), play_sound)

url_done = (
    "https://sound.peal.io/ps/audios/000/000/537/original/woo_vu_luvub_dub_dub.wav"
)
url_done = "https://www.myinstants.com/media/sounds/taco-bell-bong-sfx.mp3"
url_done = "https://www.myinstants.com/media/sounds/magic_immune.mp3"
# url_done="https://www.myinstants.com/media/sounds/tindeck_1.mp3"
# url_done="https://www.myinstants.com/media/sounds/dun_dun_1.mp3"


def allDone():
    display(
        Audio(
            url=url_done,
            autoplay=True,
        )
    )

In [3]:
data_dir = os.path.join(os.getcwd(), "data")
drone_dir = os.path.join(data_dir, "tir_pavel")

In [89]:
video0 = np.load(os.path.join(drone_dir, "pavel0.npy"))
print(video0.shape)

(1144, 436, 496)


In [5]:
import datetime as dt

timestamp0 = np.load(os.path.join(drone_dir, "timestamp0.npy")).squeeze()
print(video0.shape)


def md(matlab_datenum):
    day = dt.datetime.fromordinal(int(matlab_datenum))
    dayfrac = dt.timedelta(days=matlab_datenum % 1) - dt.timedelta(days=366)
    return pd.Timestamp(day + dayfrac).round("S")


matlab_to_date = np.vectorize(md)
timestamp0 = matlab_to_date(timestamp0)

(1144, 436, 496)


In [6]:
video0_df = pd.DataFrame({"frame": [*video0]})
video0_df.index = timestamp0
video0_df

,frame
2019-08-06 12:29:34,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,..."
2019-08-06 12:29:35,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,..."
2019-08-06 12:29:36,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,..."
2019-08-06 12:29:37,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,..."
2019-08-06 12:29:38,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,..."
...,...
2019-08-06 12:48:33,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,..."
2019-08-06 12:48:34,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,..."
2019-08-06 12:48:35,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,..."
2019-08-06 12:48:36,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,..."


#### merging the two:

making the data more coarse: 
* https://stackoverflow.com/a/39952846/6293070
* https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.resample.html 
* we probably want to resample down to half seconds? and take that average?

merging the two:
* https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge_asof.html#pandas.merge_asof
* https://stackoverflow.com/a/43979285/6293070
    

In [7]:
label_df_1 = pd.read_hdf(os.path.join(drone_dir, "pavel_label_1"), "/df")
label_df_2 = pd.read_hdf(os.path.join(drone_dir, "pavel_label_2"), "/df")
label_df = pd.concat([label_df_1, label_df_2], axis=0)
print(label_df.shape)

(503881, 29)


In [8]:
label_df = label_df.resample("1S").median()
print(label_df.shape)

(1924190, 29)


In [9]:
print(pd.Timedelta("5S"))
print(pd.Timedelta("500L"))

0 days 00:00:05
0 days 00:00:00.500000


In [10]:
video0_df_w_labels = pd.merge_asof(
    video0_df,
    label_df,
    left_index=True,
    right_index=True,
    tolerance=pd.Timedelta("3S"),
    direction="nearest",
)

print(video0_df_w_labels.shape)
print(video0_df_w_labels.head(10))

(1144, 30)
                                                                 frame  \
2019-08-06 12:29:34  [[nan, nan, nan, nan, nan, nan, nan, nan, nan,...   
2019-08-06 12:29:35  [[nan, nan, nan, nan, nan, nan, nan, nan, nan,...   
2019-08-06 12:29:36  [[nan, nan, nan, nan, nan, nan, nan, nan, nan,...   
2019-08-06 12:29:37  [[nan, nan, nan, nan, nan, nan, nan, nan, nan,...   
2019-08-06 12:29:38  [[nan, nan, nan, nan, nan, nan, nan, nan, nan,...   
2019-08-06 12:29:39  [[nan, nan, nan, nan, nan, nan, nan, nan, nan,...   
2019-08-06 12:29:40  [[nan, nan, nan, nan, nan, nan, nan, nan, nan,...   
2019-08-06 12:29:41  [[nan, nan, nan, nan, nan, nan, nan, nan, nan,...   
2019-08-06 12:29:42  [[nan, nan, nan, nan, nan, nan, nan, nan, nan,...   
2019-08-06 12:29:43  [[nan, nan, nan, nan, nan, nan, nan, nan, nan,...   

                            0         1        2         3         4  \
2019-08-06 12:29:34  15.47190  303.8815  19.6952  97.70690 -0.913253   
2019-08-06 12:29:35  15.47165 

In [11]:
any(video0_df_w_labels[23] == video0_df_w_labels[23].shift(1))

False

Somehow take the center of this:

for now just take width and height from shape
divide it by four

then take the middle half

In [131]:
# might not be the right order!

def get_middle(a):
    video0 = a
    frames, height, width = video0.shape

    cut_off = 4

    left = width // cut_off
    right = width - left
    top = height // cut_off
    bottom = height - top

    video0 = video0[:, top:bottom, left:right]
    return video0

In [132]:
def replace_nan_with_mean(a):
    return np.where(np.isnan(a), np.ones_like(a) * np.nanmean(a), a)

detrending:
* take 30 seconds of video
* wow so detrend...
* for every frame find median value
* everything above median is white (1), everything below is black(0)

In [157]:
from scipy import signal
from functools import partial

video0 = np.load(os.path.join(drone_dir, "pavel0.npy"))
print(video0.shape)

video0 = get_middle(video0)
video0 = replace_nan_with_mean(video0)

# video_yeet = scipy.signal.detrend(video0, axis=0, bp=range(0,video0.shape[0],30))
# video_yeet = scipy.signal.detrend(video0, axis=0)
video_yeet = video0

# print(video0)

def median_mask(b):
    a = b
    std = np.std(a) / 2
    a[a < np.median(a) - std] = 0
    a[a >= np.median(a) + std] = 0
    a[a != 0] = 1
    return a
    
def apply_to_outer_axis(a, func, args=[]):
    b = a
    for i in range(b.shape[0]):
        b[i] = func(b[i], *args)
    return b

# video_boi = apply_to_outer_axis(video_yeet, median_mask)
video_boi = video_yeet

print(video_boi.shape)
# print(video_boi)

(1144, 436, 496)
(1144, 218, 248)


In [158]:
def Tensor_to_video(Tensor, path):
    # try l2 norm and then min_max
    Tensor = tf.where(
        tf.math.is_nan(Tensor),
        tf.ones_like(Tensor) * np.nanmean(Tensor.numpy()),
        Tensor,
    )
    video = tf.cast(255 * min_max_norm(Tensor), tf.uint8)
    frames, width, height = video.shape
    _fourcc = cv2.VideoWriter_fourcc(*"MP4V")
    out = cv2.VideoWriter(path + ".mp4", _fourcc, 10, (width, height))
    for i in range(frames):
        img = video[i, :, :]
        img = tf.repeat(tf.expand_dims(tf.transpose(img), axis=-1), 3, axis=2).numpy()
        out.write(img)


def min_max_norm(Tensor):
    min = tf.reduce_min(Tensor)
    max = tf.reduce_max(Tensor)
    return (Tensor - min) / (max - min)


video_path = os.path.join(data_dir, "videos", "center_video")
Tensor_to_video(tf.convert_to_tensor(video_boi), video_path)

label = np.load(os.path.join(drone_dir, "pavel0.npy"))